**Titanic_Score : 0.78947**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

In [ ]:
df = pd.read_csv("../input/titanic/train.csv")
print(df.head())
print(df.columns)

In [ ]:
print(df.isna().any()) #Checking missing values

In [ ]:
#Filling the missing values in Age 
imp = SimpleImputer(strategy="mean")
df['Age'] = pd.DataFrame(imp.fit_transform(df[['Age']]))



In [ ]:
#Scaling the data
df['Sex']= df['Sex'].replace({'male': 0})
df['Sex']= df['Sex'].replace({'female': 1})
df = pd.get_dummies(df,columns=['Pclass'])
df[['Age','Fare']] = StandardScaler().fit_transform(df[['Age','Fare']])


In [ ]:
X = df.drop(columns =['PassengerId','Name','Ticket','SibSp', 'Parch','Survived','Cabin','Embarked'],axis = 1)
y = df['Survived']


X_train, X_valid, y_train, y_valid = train_test_split(X,y, train_size=0.8)

In [ ]:
## Preparing the model
model = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=90,max_leaf_nodes=120
                              ,verbose=1)
model.fit(X_train,y_train)
preds=model.predict(X_valid)

In [ ]:
cnf_matrix = metrics.confusion_matrix(y_valid, preds)
cnf_matrix


In [ ]:
#Visualization
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_valid, preds))

In [ ]:
## Importing the test data
##
##
df_test = pd.read_csv('../input/titanic/test.csv')

In [ ]:
X = df_test.drop(columns=['PassengerId','Name','Ticket','Parch','SibSp','Cabin','Embarked'],axis = 1)

#Same preprocessing on test set

imp = SimpleImputer(strategy="most_frequent")
X = pd.DataFrame(imp.fit_transform(X))
X.columns = ['Pclass','Sex','Age','Fare']
X['Sex']= X['Sex'].replace({'male': 0})
X['Sex']= X['Sex'].replace({'female': 1})

X = pd.get_dummies(X,columns=['Pclass'])
X[['Age','Fare']] = StandardScaler().fit_transform(X[['Age','Fare']])

In [ ]:
preds=model.predict(X)
preds

output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': preds})
output.to_csv('submission.csv', index=False)

In [ ]:
#thank you